In [66]:
import pandas as pd
import numpy
from __future__ import division

In [67]:
df3_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AF_segment3_workflowS4_has_duration.csv",engine='python')

In [68]:
df3_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,70289|574074|874352,Release Offer,Release Offer,24/1/2020 16:08:02,CORDELLE,Academic Services,4,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,N,3 days 4 hours 43 minutes 40 seconds,276220.0


In [69]:
numOfWorkflowProcesses = df3_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [70]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [71]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [72]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [73]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [74]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS4T[df3_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")


In [75]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [76]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step)

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [77]:
s3_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s3_t4['Num of Occurrences per step(descending)'] = list_step_count_per_key
s3_t4['Average time spends per step(descending)'] = list_step_ave_time_per_key
s3_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step,Num of Occurrences per step(descending),Average time spends per step(descending)
0,1,1,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Release Offer},"[(Release Offer, 1)]","[(Release Offer, 0.0 days 0.0 hours 0.0 minute..."
1,2,8,28 days 19 hours 41 minutes 19 seconds,3.0 days 1.0 hours 43.0 minutes 19.5 seconds,87.5%,"{Defer, Decline, Accept, Release Offer}","[(Release Offer, 8), (Accept, 5), (Defer, 2), ...","[(Release Offer, 28.0 days 19.0 hours 41.0 min..."
2,3,2,64 days 21 hours 56 minutes 37 seconds,64.0 days 21.0 hours 56.0 minutes 37.5 seconds,100.0%,"{Assessment Decision - Qualified, Manage Accep...","[(Release Offer, 2), (Accept, 2), (Assessment ...","[(Assessment Decision - Qualified, 126.0 days ..."


In [78]:
s3_t4.to_csv('engineering_AF_segment3_workflowS4_table.csv', index=False)


